# Image Classifier
_Ucadity Intro to Deep Learning Project_

In [ ]:
# Imports here
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
import torch.nn.functional as F
!pip install helper
import helper
import json
from workspace_utils import active_session
from collections import OrderedDict

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
tune_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Define the transforms for the training, tuning, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(p=0.25),
                                      transforms.RandomVerticalFlip(p=0.25),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
tune_data = datasets.ImageFolder(tune_dir, transform=test_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=30, shuffle=True)
tuneloader = torch.utils.data.DataLoader(tune_data, batch_size=30, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=30, shuffle=True)

In [ ]:
# label for classification
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
# -- BUILD THE MODEL
# Run on GPU if possible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

# Load the pre-built and pre-trained model
model = models.vgg16(pretrained=True)

# Freeze the parameters for the pre-trained model, so we dont backprop through them
for param in model.parameters():
    param.requires_grad = False
    
# Define model
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 2500)),
    ('relu1', nn.ReLU()),
    ('do1', nn.Dropout(0.2)),
    ('fc2', nn.Linear(2500, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))

# Define the model to be trained
model.classifier = classifier # the model pre-trained model VGG doesnt have .fc but .classifier

# Define loss function
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
 
# Send model to device
model.to(device)

In [ ]:
# -- TRAIN THE MODEL

# For long running code keep the session active
with active_session():
    
    # Def of variables
    epochs = 2 # nr of trainings
    steps = 0
    running_loss = 0
    print_every = 5
    train_losses, tune_losses = [], []
    
    # Training
    for epoch in range(epochs): # nr of traning times
        
        for inputs, labels in tuneloader: # loop through the data
            
            steps += 1 # train steps
            
            # Move input and label tensors to the default device (GPU if available)
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero the gradients - very important
            optimizer.zero_grad()
            
            # Forward pass -> log probabilities
            logps = model.forward(inputs)
            
            # Define loss
            loss = criterion(logps, labels)
            
            # Backward pass
            loss.backward()
            
            # Take a step towards lower loss
            optimizer.step()
            
            # Keep track on the total loss
            running_loss += loss.item()
            
            # Tune the model every print_every 5 times
            if steps % print_every == 0:
                tune_loss = 0
                accuracy = 0
                model.eval() # set model in evaluation mode
                
                with torch.no_grad(): # reduces memory usage - we dont need to calculate the gradients in evaluation mode
                    
                    for images, labels in tuneloader:
                        
                        # Move images and labels tensors to the default device (GPU if available)
                        images = images.to(device)
                        labels = labels.to(device)
                        
                        # Calculate the loss
                        log_ps = model(images) # log of probability
                        loss = criterion(log_ps, labels)
                        tune_loss += loss.item()

                        # calculate the accuracy
                        ps = torch.exp(log_ps) # get the actual probability
                        top_p, top_class = ps.topk(1, dim=1) # top probabilities and classes
                        
                        equals = top_class == labels.view(*top_class.shape) # check how many images which have the correct classification. 
                        # That *top_class.shape is passing all of the items in the top_class.shape into the view function call as separate arguments, without us even needing to know how many arguments are in the list. For example if the top_class has a shape say (32, 1), so *top_class.shape will pass 32, 1 to view function. Now this is as good as passing (top_class.shape[0],top_class.shape[1]). labels.view(top_class.shape[0],top_class.shape[1]) is equal to labels.view(*top_class.shape). The labels.view(*top_class.shape) is very useful because we dont have to know the exact shape for us to reshape them, we just use * and it will pack everything into a list and pass on to the view function for reshaping

                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                        # after getting the top_class of the top_probability from torch.exp(output), we equal the top_class with the images labels (targets) to check if they match or not. The result from this equality is binary values [0,1]. Does this mean that 1 refers to class-label matching and 0 refers to class-label mismatching? Yes

                train_losses.append(running_loss/len(trainloader))
                tune_losses.append(tune_loss/len(tuneloader))        
                        
                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Tune loss: {tune_loss/len(tuneloader):.3f}.. "
                      f"Tune accuracy: {accuracy/len(tuneloader):.3f}")
            
                running_loss = 0
                model.train()


In [ ]:
# Visualize the tuning 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.plot(train_losses, label='Training loss')
plt.plot(tune_losses, label='Tuning loss')
plt.legend(frameon=False)

In [ ]:
# -- TEST THE MODEL

with torch.no_grad(): # reduces memory usage - we dont need to calculate the gradients in evaluation mode

    # Test the network
    model.eval() # set the model into testing mode
    dataiter = iter(testloader) # make testloader iterable
    images, labels = dataiter.next() # The images you get from dataiter.next() is a batch of 64 images.
    img = images[0] # ?
    
    # Convert the 3D images to a 1D vector
    img = img.view(1, images[1]*images[2])
    
    # Calculate the probabilities (softmax) for img
    output = model.forward(img)
    ps = torch.exp(output)
    
    # Plot the image and probabilities
    helper.view_classify(img.view(1, 224, 224), ps)
    import matplotlib.pyplot as plt
    import numpy as np